# CH01-03: 開發環境測試

**課程目標:**
- 學會檢測開發環境是否正確設定
- 掌握 GPU 環境的測試方法
- 了解常見問題的診斷與排除
- 建立環境測試自動化腳本

**學習時間:** 約 60 分鐘

**前置知識:**
- Poetry 基礎操作
- NLP 套件安裝
- Python 基礎語法

---

## 📚 目錄

1. [環境檢測清單](#1)
2. [系統資訊驗證](#2)
3. [套件版本檢查](#3)
4. [GPU 環境測試](#4)
5. [NLP 工具功能測試](#5)
6. [效能基準測試](#6)
7. [問題診斷與排除](#7)
8. [自動化測試腳本](#8)

---

In [ ]:
# 環境設定與套件導入
import sys
import os
import platform
import subprocess
from importlib.metadata import version, PackageNotFoundError
from pathlib import Path
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定顯示風格
sns.set_style('whitegrid')

print("✅ 環境設定完成")
print(f"Python 版本: {sys.version.split()[0]}")
print(f"系統平台: {platform.system()} {platform.release()}")

<a id="1"></a>
## 1. 環境檢測清單

### 1.1 環境測試的層次架構

NLP 開發環境測試可分為四個層次:

1. **系統層 (System Level)** - 作業系統、Python 版本、記憶體
2. **套件層 (Package Level)** - 所有依賴套件版本檢查
3. **硬體層 (Hardware Level)** - GPU、CUDA、cuDNN 驗證
4. **功能層 (Functional Level)** - NLP 工具功能測試

---

In [ ]:
# 環境檢測清單視覺化
checklist_data = {
    '檢測層次': [
        '系統層',
        '系統層',
        '系統層',
        '套件層',
        '套件層',
        '硬體層',
        '硬體層',
        '功能層',
        '功能層'
    ],
    '檢測項目': [
        'OS 版本',
        'Python 版本',
        '可用記憶體',
        '核心套件版本',
        '依賴完整性',
        'CUDA 可用性',
        'GPU 記憶體',
        '分詞功能',
        '模型載入'
    ],
    '檢測方法': [
        'platform.system()',
        'sys.version',
        'psutil.virtual_memory()',
        'importlib.metadata.version()',
        'poetry show',
        'torch.cuda.is_available()',
        'torch.cuda.get_device_properties()',
        'jieba.cut() / word_tokenize()',
        'AutoModel.from_pretrained()'
    ],
    '重要性': ['⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', 
              '⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐']
}

df_checklist = pd.DataFrame(checklist_data)

fig, ax = plt.subplots(figsize=(16, 8))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_checklist.values, colLabels=df_checklist.columns,
                cellLoc='left', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.2)

# 設定表頭樣式
for i in range(len(df_checklist.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 設定層次顏色
colors = {
    '系統層': '#E8F4F8',
    '套件層': '#FFF9E6',
    '硬體層': '#FFE6E6',
    '功能層': '#E6F7E6'
}

for i in range(1, len(df_checklist) + 1):
    layer = df_checklist.iloc[i-1]['檢測層次']
    for j in range(len(df_checklist.columns)):
        table[(i, j)].set_facecolor(colors[layer])

plt.title('環境檢測清單', fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📋 環境檢測流程:")
print("  1️⃣ 系統層 → 2️⃣ 套件層 → 3️⃣ 硬體層 → 4️⃣ 功能層")
print("  由底層到應用層,逐步驗證環境完整性")

<a id="2"></a>
## 2. 系統資訊驗證

### 2.1 作業系統與 Python 環境

In [ ]:
# 系統資訊檢測
def check_system_info():
    """檢查系統基本資訊"""
    
    print("="*70)
    print("  系統資訊檢測")
    print("="*70)
    
    system_info = {
        '作業系統': f"{platform.system()} {platform.release()}",
        'CPU 架構': platform.machine(),
        'Python 版本': sys.version.split()[0],
        'Python 執行檔': sys.executable,
        '工作目錄': os.getcwd(),
    }
    
    for key, value in system_info.items():
        print(f"  {key:15s}: {value}")
    
    # 檢查虛擬環境
    in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
    
    if in_venv:
        print(f"\n  ✅ 正在虛擬環境中")
        print(f"  環境路徑: {sys.prefix}")
    else:
        print(f"\n  ⚠️ 未在虛擬環境中 (使用系統 Python)")
        print(f"  建議: poetry shell")
    
    print("\n" + "="*70)

# 執行檢查
check_system_info()

### 2.2 記憶體資訊檢查

In [ ]:
# 記憶體資訊檢查
def check_memory():
    """檢查系統記憶體資訊"""
    
    print("="*70)
    print("  記憶體資訊")
    print("="*70)
    
    try:
        import psutil
        
        memory = psutil.virtual_memory()
        
        total_gb = memory.total / (1024**3)
        available_gb = memory.available / (1024**3)
        used_gb = memory.used / (1024**3)
        
        print(f"  總記憶體:   {total_gb:.2f} GB")
        print(f"  已使用:     {used_gb:.2f} GB")
        print(f"  可用:       {available_gb:.2f} GB")
        print(f"  使用率:     {memory.percent}%")
        
        # 視覺化記憶體使用
        fig, ax = plt.subplots(figsize=(10, 6))
        
        sizes = [used_gb, available_gb]
        labels = [f'已使用\n{used_gb:.1f} GB', f'可用\n{available_gb:.1f} GB']
        colors = ['#FF6B6B', '#4ECDC4']
        explode = (0.05, 0)
        
        ax.pie(sizes, explode=explode, labels=labels, colors=colors,
              autopct='%1.1f%%', shadow=True, startangle=90,
              textprops={'fontsize': 12, 'weight': 'bold'})
        
        ax.set_title(f'系統記憶體使用狀況 (總計: {total_gb:.1f} GB)', 
                    fontsize=16, fontweight='bold', pad=20)
        
        plt.tight_layout()
        plt.show()
        
        # 建議
        if memory.percent > 80:
            print("\n  ⚠️ 記憶體使用率過高 (>80%)")
            print("  建議: 關閉不必要的應用程式")
        elif available_gb < 2:
            print("\n  ⚠️ 可用記憶體不足 (<2GB)")
            print("  建議: 訓練大型模型可能遇到問題")
        else:
            print("\n  ✅ 記憶體充足,可正常運行 NLP 任務")
        
    except ImportError:
        print("  ⚠️ psutil 未安裝,無法檢測記憶體")
        print("  安裝: poetry add psutil")
    
    print("\n" + "="*70)

# 執行檢查
check_memory()

<a id="3"></a>
## 3. 套件版本檢查

### 3.1 核心套件版本驗證

In [ ]:
# 核心套件版本檢查
def check_package_versions():
    """檢查所有核心套件版本"""
    
    packages = [
        # 基礎工具
        ('numpy', '1.24.0'),
        ('pandas', '2.0.0'),
        ('scikit-learn', '1.3.0'),
        ('matplotlib', '3.7.0'),
        # 中文 NLP
        ('jieba', None),
        # 英文 NLP
        ('nltk', '3.8.0'),
        ('spacy', '3.7.0'),
        # 深度學習
        ('torch', '2.0.0'),
        ('tensorflow', '2.10.0'),
        # Transformers 生態
        ('transformers', '4.30.0'),
        ('datasets', None),
        ('tokenizers', None),
    ]
    
    print("="*70)
    print("  核心套件版本檢查")
    print("="*70)
    
    results = []
    
    for package, min_version in packages:
        try:
            pkg_version = version(package)
            status = "✅"
            note = "OK"
            
            if min_version:
                # 簡單版本比較 (實際應使用 packaging.version)
                if pkg_version < min_version:
                    status = "⚠️"
                    note = f"建議 >= {min_version}"
            
            print(f"  {status} {package:20s} {pkg_version:15s} {note}")
            results.append({'package': package, 'installed': True, 'version': pkg_version})
            
        except PackageNotFoundError:
            print(f"  ❌ {package:20s} {'未安裝':15s} poetry add {package}")
            results.append({'package': package, 'installed': False, 'version': None})
    
    # 統計
    installed = sum(1 for r in results if r['installed'])
    total = len(results)
    
    print("\n" + "="*70)
    print(f"  套件安裝狀態: {installed}/{total} 已安裝 ({installed/total*100:.1f}%)")
    
    if installed == total:
        print("\n  ✅ 所有核心套件已正確安裝")
    else:
        print("\n  ⚠️ 部分套件缺失,請根據上方提示安裝")
    
    print("="*70)
    
    return results

# 執行檢查
package_results = check_package_versions()

In [ ]:
# 視覺化套件安裝狀態
if package_results:
    installed_count = sum(1 for r in package_results if r['installed'])
    missing_count = len(package_results) - installed_count
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 圓餅圖
    sizes = [installed_count, missing_count]
    labels = [f'已安裝\n{installed_count}', f'未安裝\n{missing_count}']
    colors = ['#4ECDC4', '#FF6B6B']
    explode = (0.05, 0.05)
    
    axes[0].pie(sizes, explode=explode, labels=labels, colors=colors,
               autopct='%1.1f%%', shadow=True, startangle=90,
               textprops={'fontsize': 12, 'weight': 'bold'})
    axes[0].set_title('套件安裝狀態', fontsize=14, fontweight='bold')
    
    # 長條圖 (分類統計)
    categories = ['基礎工具', '中文 NLP', '英文 NLP', '深度學習', 'Transformers']
    category_ranges = [
        (0, 4),   # 基礎工具
        (4, 5),   # 中文 NLP
        (5, 7),   # 英文 NLP
        (7, 9),   # 深度學習
        (9, 12)   # Transformers
    ]
    
    category_installed = []
    for start, end in category_ranges:
        count = sum(1 for r in package_results[start:end] if r['installed'])
        category_installed.append(count)
    
    bars = axes[1].bar(categories, category_installed, color='#4ECDC4')
    axes[1].set_ylabel('已安裝套件數', fontsize=12)
    axes[1].set_title('各類別套件安裝狀態', fontsize=14, fontweight='bold')
    axes[1].set_ylim(0, max([r[1]-r[0] for r in category_ranges]) + 1)
    
    # 標註數值
    for bar, total_range in zip(bars, category_ranges):
        height = bar.get_height()
        total = total_range[1] - total_range[0]
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}/{total}',
                    ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

<a id="4"></a>
## 4. GPU 環境測試

### 4.1 PyTorch GPU 檢測

In [ ]:
# PyTorch GPU 環境測試
def test_pytorch_gpu():
    """測試 PyTorch GPU 可用性"""
    
    print("="*70)
    print("  PyTorch GPU 環境測試")
    print("="*70)
    
    try:
        import torch
        
        print(f"  PyTorch 版本: {torch.__version__}")
        print(f"  CUDA 編譯版本: {torch.version.cuda}")
        print(f"  cuDNN 版本: {torch.backends.cudnn.version()}")
        print(f"  CUDA 可用: {torch.cuda.is_available()}")
        
        if torch.cuda.is_available():
            print(f"\n  ✅ GPU 環境可用")
            print(f"  GPU 數量: {torch.cuda.device_count()}")
            
            for i in range(torch.cuda.device_count()):
                props = torch.cuda.get_device_properties(i)
                print(f"\n  GPU {i}: {torch.cuda.get_device_name(i)}")
                print(f"    記憶體: {props.total_memory / 1e9:.2f} GB")
                print(f"    計算能力: {props.major}.{props.minor}")
            
            # GPU 計算測試
            print("\n  🧪 GPU 計算測試:")
            x = torch.rand(1000, 1000).cuda()
            y = torch.rand(1000, 1000).cuda()
            
            start = time.time()
            z = torch.matmul(x, y)
            torch.cuda.synchronize()
            gpu_time = time.time() - start
            
            print(f"    矩陣乘法 (1000x1000): {gpu_time*1000:.2f} ms")
            print(f"    結果形狀: {z.shape}")
            
            # GPU 記憶體使用
            allocated = torch.cuda.memory_allocated() / 1e6
            cached = torch.cuda.memory_reserved() / 1e6
            print(f"\n  GPU 記憶體使用:")
            print(f"    已分配: {allocated:.2f} MB")
            print(f"    已快取: {cached:.2f} MB")
            
        else:
            print("\n  ⚠️ CUDA 不可用,請檢查:")
            print("    1. NVIDIA 驅動是否安裝: nvidia-smi")
            print("    2. PyTorch 是否為 GPU 版本")
            print("    3. CUDA 版本是否匹配")
        
    except ImportError:
        print("  ❌ PyTorch 未安裝")
        print("  安裝: poetry add torch")
    except Exception as e:
        print(f"  ❌ GPU 測試失敗: {e}")
    
    print("\n" + "="*70)

# 執行測試
test_pytorch_gpu()

### 4.2 TensorFlow GPU 檢測

In [ ]:
# TensorFlow GPU 環境測試
def test_tensorflow_gpu():
    """測試 TensorFlow GPU 可用性"""
    
    print("="*70)
    print("  TensorFlow GPU 環境測試")
    print("="*70)
    
    try:
        import tensorflow as tf
        
        print(f"  TensorFlow 版本: {tf.__version__}")
        
        # GPU 裝置列表
        gpus = tf.config.list_physical_devices('GPU')
        print(f"  GPU 裝置數量: {len(gpus)}")
        
        if len(gpus) > 0:
            print("\n  ✅ GPU 環境可用")
            for gpu in gpus:
                print(f"    - {gpu}")
            
            # GPU 計算測試
            print("\n  🧪 GPU 計算測試:")
            with tf.device('/GPU:0'):
                x = tf.random.normal([1000, 1000])
                y = tf.random.normal([1000, 1000])
                
                start = time.time()
                z = tf.matmul(x, y)
                tf_time = (time.time() - start) * 1000
                
                print(f"    矩陣乘法 (1000x1000): {tf_time:.2f} ms")
                print(f"    結果形狀: {z.shape}")
        else:
            print("\n  ⚠️ TensorFlow 偵測不到 GPU")
            print("    檢查項目:")
            print("    1. TensorFlow 版本是否支援 GPU")
            print("    2. CUDA/cuDNN 版本是否相容")
            print("    3. 安裝 GPU 版本: pip install tensorflow[and-cuda]")
        
    except ImportError:
        print("  ⚠️ TensorFlow 未安裝 (可選)")
        print("  安裝: poetry add tensorflow")
    except Exception as e:
        print(f"  ❌ GPU 測試失敗: {e}")
    
    print("\n" + "="*70)

# 執行測試
test_tensorflow_gpu()

### 4.3 NVIDIA 驅動檢查

In [ ]:
# NVIDIA 驅動檢查
def check_nvidia_driver():
    """檢查 NVIDIA 驅動狀態"""
    
    print("="*70)
    print("  NVIDIA 驅動檢測")
    print("="*70)
    
    try:
        result = subprocess.run(['nvidia-smi'], 
                              capture_output=True, text=True, check=True)
        
        if result.returncode == 0:
            print("\n  ✅ NVIDIA 驅動已安裝\n")
            # 顯示 nvidia-smi 輸出
            print(result.stdout)
        else:
            print("  ❌ nvidia-smi 執行失敗")
            
    except FileNotFoundError:
        print("\n  ❌ nvidia-smi 未找到")
        print("\n  可能原因:")
        print("    1. 未安裝 NVIDIA 驅動")
        print("    2. 非 NVIDIA GPU 系統")
        print("    3. 驅動未正確加入 PATH")
        print("\n  解決方案:")
        print("    - 下載驅動: https://www.nvidia.com/drivers")
        print("    - 或使用 CPU 版本的深度學習框架")
    except Exception as e:
        print(f"  ❌ 檢測失敗: {e}")
    
    print("\n" + "="*70)

# 執行檢查
check_nvidia_driver()

<a id="5"></a>
## 5. NLP 工具功能測試

### 5.1 中文分詞測試 (jieba)

In [ ]:
# jieba 功能測試
def test_jieba():
    """測試 jieba 分詞功能"""
    
    print("="*70)
    print("  jieba 分詞測試")
    print("="*70)
    
    try:
        import jieba
        
        test_text = "我愛自然語言處理,深度學習很有趣"
        
        print(f"\n  測試文本: {test_text}")
        
        # 精確模式
        result_precise = list(jieba.cut(test_text))
        print(f"  精確模式: {result_precise}")
        
        # 全模式
        result_full = list(jieba.cut(test_text, cut_all=True))
        print(f"  全模式:   {result_full}")
        
        # 搜尋引擎模式
        result_search = list(jieba.cut_for_search(test_text))
        print(f"  搜尋模式: {result_search}")
        
        print("\n  ✅ jieba 分詞功能正常")
        
    except ImportError:
        print("\n  ❌ jieba 未安裝")
        print("  安裝: poetry add jieba")
    except Exception as e:
        print(f"\n  ❌ 測試失敗: {e}")
    
    print("\n" + "="*70)

# 執行測試
test_jieba()

### 5.2 英文 NLP 工具測試

In [ ]:
# NLTK 與 spaCy 功能測試
def test_english_nlp():
    """測試英文 NLP 工具"""
    
    print("="*70)
    print("  英文 NLP 工具測試")
    print("="*70)
    
    # NLTK 測試
    print("\n  【NLTK】")
    try:
        from nltk.tokenize import word_tokenize
        
        text = "Natural language processing is amazing!"
        tokens = word_tokenize(text)
        print(f"    測試文本: {text}")
        print(f"    分詞結果: {tokens}")
        print(f"    ✅ NLTK 分詞正常")
        
    except LookupError:
        print("    ❌ NLTK 數據包未下載")
        print("    執行: nltk.download('punkt')")
    except ImportError:
        print("    ❌ NLTK 未安裝")
    except Exception as e:
        print(f"    ❌ 測試失敗: {e}")
    
    # spaCy 測試
    print("\n  【spaCy】")
    try:
        import spacy
        
        nlp = spacy.load('en_core_web_sm')
        text = "Apple is looking at buying U.K. startup"
        doc = nlp(text)
        
        print(f"    測試文本: {text}")
        
        # 實體識別
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        print(f"    實體識別: {entities}")
        
        # 詞性標註
        pos_tags = [(token.text, token.pos_) for token in doc[:3]]
        print(f"    詞性標註: {pos_tags}")
        print(f"    ✅ spaCy 功能正常")
        
    except OSError:
        print("    ❌ spaCy 模型未下載")
        print("    執行: python -m spacy download en_core_web_sm")
    except ImportError:
        print("    ❌ spaCy 未安裝")
    except Exception as e:
        print(f"    ❌ 測試失敗: {e}")
    
    print("\n" + "="*70)

# 執行測試
test_english_nlp()

### 5.3 Transformers 模型載入測試

In [ ]:
# Transformers 模型測試
def test_transformers():
    """測試 Transformers 模型載入"""
    
    print("="*70)
    print("  Transformers 模型測試")
    print("="*70)
    
    try:
        from transformers import AutoTokenizer, AutoModel
        
        print("\n  ✅ Transformers 模組載入成功")
        
        # 測試小型模型 (避免長時間下載)
        print("\n  🧪 測試模型載入 (distilbert-base-uncased):")
        
        try:
            tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
            print("    ✅ Tokenizer 載入成功")
            
            # 簡單測試
            text = "Hello, this is a test."
            tokens = tokenizer.tokenize(text)
            print(f"    測試文本: {text}")
            print(f"    Token 化: {tokens}")
            
        except Exception as e:
            print(f"    ⚠️ 模型下載可能需要時間或網路連線")
            print(f"    錯誤: {e}")
        
    except ImportError:
        print("\n  ❌ Transformers 未安裝")
        print("  安裝: poetry add transformers")
    except Exception as e:
        print(f"\n  ❌ 測試失敗: {e}")
    
    print("\n" + "="*70)

# 執行測試 (取消註解,首次可能需要下載模型)
# test_transformers()

print("💡 取消註解執行 Transformers 測試 (首次會下載模型)")

<a id="6"></a>
## 6. 效能基準測試

### 6.1 CPU vs GPU 效能對比

In [ ]:
# CPU vs GPU 效能基準測試
def benchmark_cpu_gpu():
    """CPU 與 GPU 效能對比測試"""
    
    print("="*70)
    print("  CPU vs GPU 效能基準測試")
    print("="*70)
    
    try:
        import torch
        import numpy as np
        
        size = 5000
        
        # NumPy (CPU) 測試
        print("\n  【NumPy CPU 測試】")
        a = np.random.rand(size, size)
        b = np.random.rand(size, size)
        
        start = time.time()
        c = np.dot(a, b)
        cpu_time = time.time() - start
        
        print(f"    矩陣大小: {size} x {size}")
        print(f"    計算時間: {cpu_time:.4f} 秒")
        
        # PyTorch GPU 測試
        if torch.cuda.is_available():
            print("\n  【PyTorch GPU 測試】")
            
            device = torch.device('cuda')
            x = torch.rand(size, size, device=device)
            y = torch.rand(size, size, device=device)
            
            torch.cuda.synchronize()
            start = time.time()
            z = torch.matmul(x, y)
            torch.cuda.synchronize()
            gpu_time = time.time() - start
            
            print(f"    矩陣大小: {size} x {size}")
            print(f"    計算時間: {gpu_time:.4f} 秒")
            print(f"\n  📊 加速比: {cpu_time / gpu_time:.2f}x")
            
            # 視覺化對比
            plt.figure(figsize=(10, 6))
            
            devices = ['CPU\n(NumPy)', 'GPU\n(PyTorch)']
            times = [cpu_time, gpu_time]
            colors = ['#FF6B6B', '#4ECDC4']
            
            bars = plt.bar(devices, times, color=colors)
            plt.ylabel('計算時間 (秒)', fontsize=12)
            plt.title(f'矩陣乘法效能對比 ({size}x{size})', fontsize=14, fontweight='bold')
            
            # 標註時間
            for bar, t in zip(bars, times):
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2., height,
                        f'{t:.4f}s',
                        ha='center', va='bottom', fontsize=12, fontweight='bold')
            
            # 加速比文字
            plt.text(0.5, max(times) * 0.8, 
                    f'GPU 加速: {cpu_time/gpu_time:.1f}x',
                    ha='center', fontsize=16, fontweight='bold',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
            
            plt.tight_layout()
            plt.show()
            
        else:
            print("\n  ⚠️ GPU 不可用,無法進行對比測試")
        
    except ImportError:
        print("\n  ❌ PyTorch 未安裝")
    except Exception as e:
        print(f"\n  ❌ 測試失敗: {e}")
    
    print("\n" + "="*70)

# 執行測試 (取消註解,需要 GPU)
# benchmark_cpu_gpu()

print("💡 取消註解執行效能測試 (需要 GPU)")

<a id="7"></a>
## 7. 問題診斷與排除

### 7.1 常見問題診斷決策樹

In [ ]:
# 常見問題與解決方案
common_issues = {
    '問題': [
        'ModuleNotFoundError',
        'CUDA out of memory',
        'ImportError: libcudnn.so.8',
        'LookupError: punkt not found',
        'OSError: en_core_web_sm not found',
        'Version conflict'
    ],
    '原因': [
        '套件未安裝或虛擬環境未啟動',
        'GPU 記憶體不足',
        'cuDNN 未安裝或版本不匹配',
        'NLTK 數據包未下載',
        'spaCy 模型未下載',
        '套件版本衝突'
    ],
    '解決方案': [
        '1. poetry shell\n2. poetry install',
        '1. 減少 batch_size\n2. torch.cuda.empty_cache()',
        '1. 重裝 PyTorch (含 cuDNN)\n2. 檢查 CUDA 版本',
        "nltk.download('punkt')",
        'python -m spacy download en_core_web_sm',
        '1. poetry update\n2. poetry lock --no-update'
    ]
}

df_issues = pd.DataFrame(common_issues)

fig, ax = plt.subplots(figsize=(16, 7))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_issues.values, colLabels=df_issues.columns,
                cellLoc='left', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 3)

# 設定表頭
for i in range(len(df_issues.columns)):
    table[(0, i)].set_facecolor('#FF6B6B')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 交替行顏色
for i in range(1, len(df_issues) + 1):
    for j in range(len(df_issues.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#F0F0F0')

plt.title('常見問題診斷與解決方案', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔍 診斷流程:")
print("  1. 確認錯誤訊息關鍵字")
print("  2. 根據上表找到對應問題")
print("  3. 執行建議的解決方案")
print("  4. 重新執行測試驗證")

<a id="8"></a>
## 8. 自動化測試腳本

### 8.1 完整環境檢測腳本

In [ ]:
# 完整環境檢測腳本
def run_full_environment_test():
    """執行完整環境檢測"""
    
    print("\n" + "="*70)
    print("  NLP 開發環境完整檢測報告")
    print("="*70)
    print(f"  檢測時間: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*70)
    
    test_results = []
    
    # 測試 1: 系統資訊
    print("\n[1/6] 系統資訊檢測...")
    try:
        check_system_info()
        test_results.append(('系統資訊', '✅'))
    except:
        test_results.append(('系統資訊', '❌'))
    
    # 測試 2: 套件版本
    print("\n[2/6] 套件版本檢查...")
    try:
        results = check_package_versions()
        installed = sum(1 for r in results if r['installed'])
        if installed == len(results):
            test_results.append(('套件版本', '✅'))
        else:
            test_results.append(('套件版本', '⚠️'))
    except:
        test_results.append(('套件版本', '❌'))
    
    # 測試 3: GPU 環境
    print("\n[3/6] GPU 環境測試...")
    try:
        test_pytorch_gpu()
        test_results.append(('GPU 環境', '✅'))
    except:
        test_results.append(('GPU 環境', '⚠️'))
    
    # 測試 4: jieba
    print("\n[4/6] jieba 測試...")
    try:
        test_jieba()
        test_results.append(('jieba', '✅'))
    except:
        test_results.append(('jieba', '❌'))
    
    # 測試 5: 英文 NLP
    print("\n[5/6] 英文 NLP 工具測試...")
    try:
        test_english_nlp()
        test_results.append(('英文 NLP', '✅'))
    except:
        test_results.append(('英文 NLP', '⚠️'))
    
    # 測試 6: 記憶體
    print("\n[6/6] 記憶體檢查...")
    try:
        check_memory()
        test_results.append(('記憶體', '✅'))
    except:
        test_results.append(('記憶體', '⚠️'))
    
    # 總結報告
    print("\n" + "="*70)
    print("  檢測結果總覽")
    print("="*70)
    
    for test_name, result in test_results:
        print(f"  {result} {test_name}")
    
    # 統計
    passed = sum(1 for _, r in test_results if r == '✅')
    warning = sum(1 for _, r in test_results if r == '⚠️')
    failed = sum(1 for _, r in test_results if r == '❌')
    
    print("\n" + "="*70)
    print(f"  通過: {passed} | 警告: {warning} | 失敗: {failed}")
    
    if failed == 0 and warning == 0:
        print("\n  🎉 環境檢測全部通過,可以開始 NLP 開發!")
    elif failed == 0:
        print("\n  ⚠️ 部分項目有警告,建議檢查並修復")
    else:
        print("\n  ❌ 部分測試失敗,請根據上方提示修復問題")
    
    print("="*70)
    
    return test_results

# 執行完整測試 (取消註解)
# results = run_full_environment_test()

print("💡 取消註解執行完整環境檢測")

### 8.2 生成 HTML 測試報告

In [ ]:
# 生成 HTML 環境報告
def generate_html_report(test_results):
    """生成 HTML 格式的環境測試報告"""
    
    html_template = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>NLP 環境檢測報告</title>
        <style>
            body {{ font-family: 'Microsoft JhengHei', sans-serif; margin: 40px; }}
            h1 {{ color: #4ECDC4; }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
            th {{ background-color: #4ECDC4; color: white; }}
            tr:nth-child(even) {{ background-color: #f2f2f2; }}
            .pass {{ color: green; font-weight: bold; }}
            .warning {{ color: orange; font-weight: bold; }}
            .fail {{ color: red; font-weight: bold; }}
        </style>
    </head>
    <body>
        <h1>🔍 NLP 開發環境檢測報告</h1>
        <p><strong>檢測時間:</strong> {timestamp}</p>
        <p><strong>系統資訊:</strong> {system_info}</p>
        
        <h2>測試結果</h2>
        <table>
            <tr>
                <th>測試項目</th>
                <th>結果</th>
                <th>狀態</th>
            </tr>
            {test_rows}
        </table>
        
        <h2>總結</h2>
        <p>{summary}</p>
    </body>
    </html>
    """
    
    # 生成測試行
    test_rows = ""
    for test_name, result in test_results:
        status_class = 'pass' if result == '✅' else ('warning' if result == '⚠️' else 'fail')
        status_text = '通過' if result == '✅' else ('警告' if result == '⚠️' else '失敗')
        test_rows += f"<tr><td>{test_name}</td><td class='{status_class}'>{result}</td><td>{status_text}</td></tr>\n"
    
    # 統計
    passed = sum(1 for _, r in test_results if r == '✅')
    total = len(test_results)
    
    summary = f"通過 {passed}/{total} 項測試"
    if passed == total:
        summary += " - 環境配置完美! 🎉"
    
    # 填充模板
    html = html_template.format(
        timestamp=time.strftime('%Y-%m-%d %H:%M:%S'),
        system_info=f"{platform.system()} {platform.release()}, Python {sys.version.split()[0]}",
        test_rows=test_rows,
        summary=summary
    )
    
    # 儲存報告
    report_path = Path('environment_report.html')
    report_path.write_text(html, encoding='utf-8')
    
    print(f"\n✅ HTML 報告已生成: {report_path.absolute()}")
    print(f"   請使用瀏覽器開啟查看")

# 使用範例 (需要先執行 run_full_environment_test)
# generate_html_report(results)

print("💡 執行完整測試後,可生成 HTML 報告")
print("   1. results = run_full_environment_test()")
print("   2. generate_html_report(results)")

---

## 📚 本課總結

### 核心要點回顧:

1. **環境檢測四層架構:**
   - 系統層: OS、Python、記憶體
   - 套件層: 版本、依賴完整性
   - 硬體層: GPU、CUDA、cuDNN
   - 功能層: NLP 工具功能測試

2. **GPU 環境驗證:**
   - PyTorch: torch.cuda.is_available()
   - TensorFlow: tf.config.list_physical_devices('GPU')
   - NVIDIA 驅動: nvidia-smi

3. **NLP 工具測試:**
   - jieba: 分詞功能
   - NLTK: 需下載數據包
   - spaCy: 需下載語言模型
   - Transformers: 模型載入與推理

4. **常見問題診斷:**
   - ModuleNotFoundError → 虛擬環境未啟動
   - CUDA out of memory → 減少 batch_size
   - 數據包未找到 → 下載對應資源

5. **自動化測試:**
   - 完整環境檢測腳本
   - HTML 報告生成
   - CI/CD 整合

---

## 🎯 下節預告

**CH02-01: 什麼是自然語言處理**

我們將開始探討:
- NLP 的定義與應用場景
- NLP 技術發展史
- NLP 核心任務分類
- 中文 NLP 的特殊挑戰

---

## 📖 延伸閱讀

1. **環境配置指南:**
   - [PyTorch CUDA 文檔](https://pytorch.org/get-started/locally/)
   - [TensorFlow GPU 支援](https://www.tensorflow.org/install/gpu)
   - [NVIDIA CUDA 工具包](https://developer.nvidia.com/cuda-toolkit)

2. **診斷工具:**
   - [nvidia-smi 使用指南](https://developer.nvidia.com/nvidia-system-management-interface)
   - [PyTorch 效能分析](https://pytorch.org/tutorials/recipes/recipes/benchmark.html)

3. **最佳實踐:**
   - [環境隔離策略](https://python-poetry.org/docs/managing-environments/)
   - [CI/CD 環境測試](https://docs.github.com/en/actions)

---

### 🙋 問題討論

有任何問題嗎?歡迎在討論區提問!

---

**課程資訊:**
- **作者:** iSpan NLP Team
- **版本:** v1.0
- **最後更新:** 2025-10-17
- **授權:** MIT License (僅供教學使用)